# 📘 2주차 - Agent의 구조화된 실행과 LangChain 도입 (v2)

### 🎯 수업 목표
1.  **서로 다른 두 개의 Tool**을 순차적으로 사용해야 하는 복잡한 문제를 해결할 수 있다.
2.  이 과정을 **수동으로 구현**하며 Agent의 내부 동작 원리와 **명확한 한계**를 깊이 있게 이해한다.
3.  **LangChain** 프레임워크가 이 복잡한 과정을 어떻게 자동화하는지 배우고, 그 편리함을 체감한다.
4.  직접 **LangChain Agent**를 만들어 실용적인 문제를 해결할 수 있다.

### 1. 새로운 도전: 지식 검색과 실시간 정보 결합

지난 예제에서는 `wikipedia`라는 단일 Tool의 불안정성 때문에 어려움을 겪었습니다. 이번에는 더 안정적이고 명확한 예제를 통해 Agent의 핵심 개념을 다시 배워보겠습니다.

**오늘의 도전 과제:**
> "일본의 수도는 어디인지 위키백과에서 찾고, 그 도시의 현재 날씨를 알려줘."

이 문제는 '정적인 지식(수도)'과 '실시간 정보(날씨)'를 결합해야 하는, 더 현실적인 Agent의 사용 사례입니다.

In [1]:
#  기본 설정: 패키지 설치 및 API 키 설정
!pip install openai wikipedia langchain langchain_openai langchainhub -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00


In [2]:
import openai
import os
import json
import wikipedia

# --- OpenAI API 키를 입력하세요 --- #
### 실습에서는 dotenv를 이용해주세요
os.environ['OPENAI_API_KEY'] = 'your api key'
# ---------------------------------- #

client = openai.OpenAI()
MODEL = "gpt-4-turbo"

### 2. 두 개의 Tool 준비하기
이번 실습에서는 `위키피디아 검색` Tool과 `날씨 확인` Tool, 두 가지를 사용합니다.

In [3]:
# Tool 1: 위키피디아 검색
def search_wikipedia(query: str) -> str:
    """위키백과에서 정보를 검색하여 요약합니다."""
    print(f"[Tool 실행] search_wikipedia(query='{query}')")
    try:
        wikipedia.set_lang("ko")
        return wikipedia.summary(query, sentences=2)
    except wikipedia.exceptions.PageError:
        return f"'{query}'에 대한 위키백과 페이지를 찾을 수 없습니다."
    except Exception as e:
        return f"위키백과 검색 중 오류 발생: {e}"

# Tool 2: 현재 날씨 확인 (가상 Tool)
# 참고: 안정적인 실습을 위해 실제 API를 호출하는 대신 특정 입력에 정해진 값을 반환하는 '가상(Mock)' 함수를 사용합니다.
def get_current_weather(city: str) -> str:
    """특정 도시의 현재 날씨를 알려줍니다."""
    print(f"[Tool 실행] get_current_weather(city='{city}')")
    if "도쿄" in city or "Tokyo" in city:
        return "맑음, 28°C"
    elif "서울" in city or "Seoul" in city:
        return "흐림, 24°C"
    else:
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."

# OpenAI에 등록할 Tool 스키마 정의
wiki_tool_schema = {
    "type": "function",
    "function": {
        "name": "search_wikipedia",
        "description": "위키백과(Wikipedia)에서 특정 주제에 대한 일반적인 사실 정보를 검색합니다.",
        "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "검색할 키워드"}}, "required": ["query"]}
    }
}

weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "특정 도시의 현재 날씨를 알려줍니다.",
        "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "날씨를 알고 싶은 도시 이름"}}, "required": ["city"]}
    }
}

available_functions = {
    "search_wikipedia": search_wikipedia,
    "get_current_weather": get_current_weather
}

---
### 😫 3. 수동으로 Multi-Step Agent 구현하기

먼저, 프레임워크 없이 이 문제를 해결하는 과정을 통해 그 복잡성과 한계를 체감해 보겠습니다.

#### 3-1. 성공 시나리오 (정상 작동하는 경우)

In [8]:
print("---  수동 Agent 실행 시작 ---")
tools = [wiki_tool_schema, weather_tool_schema]

# --- 1단계: 일본의 수도 검색 --- #
print("\n--- 1단계: 수도 검색 ---")
messages = [{"role": "user", "content": "일본의 수도는 어디야?"}]
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
tool_call = response.choices[0].message.tool_calls[0]
capital_info = available_functions[tool_call.function.name](**json.loads(tool_call.function.arguments))
print(f"Tool 실행 결과 (수도 정보): {capital_info}")

# --- 2단계: 결과에서 '도시 이름'만 추출 (LLM을 파서로 활용) --- #
print("\n--- 2단계: 도시 이름 추출 ---")
messages = [
    {"role": "system", "content": "주어진 텍스트에서 도시 이름만 정확히 추출해서 알려줘. 다른 설명은 붙이지 마."},
    {"role": "user", "content": capital_info}
]
response = client.chat.completions.create(model=MODEL, messages=messages)
capital_city = response.choices[0].message.content
print(f"추출된 도시 이름: {capital_city}")

# --- 3단계: 추출된 도시 이름으로 날씨 검색 --- #
print("\n--- 3단계: 날씨 정보 검색 ---")
messages = [{"role": "user", "content": f"{capital_city}의 현재 날씨 알려줘."}]
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
tool_call = response.choices[0].message.tool_calls[0]
weather_info = available_functions[tool_call.function.name](**json.loads(tool_call.function.arguments))
print(f"Tool 실행 결과 (날씨 정보): {weather_info}")

# --- 4단계: 모든 정보를 종합하여 최종 답변 생성 --- #
print("\n--- 4단계: 최종 답변 생성 ---")
final_prompt = f"다음 정보를 종합해서 '일본의 수도와 현재 날씨'에 대해 한 문장으로 답변해줘.\n\n#수도: {capital_city}\n#날씨: {weather_info}"
messages = [{"role": "user", "content": final_prompt}]
response = client.chat.completions.create(model=MODEL, messages=messages)
final_answer = response.choices[0].message.content

print("\n===== 수동 Agent의 최종 답변 (성공) =====")
print(final_answer)

---  수동 Agent 실행 시작 ---

--- 1단계: 수도 검색 ---
[Tool 실행] search_wikipedia(query='일본의 수도')
Tool 실행 결과 (수도 정보): 현재의 일본의 수도는 도쿄이다. 이 문서는 역사속 일본의 수도(日本の首都)에 관해 기술한 것이다.

--- 2단계: 도시 이름 추출 ---
추출된 도시 이름: 도쿄

--- 3단계: 날씨 정보 검색 ---
[Tool 실행] get_current_weather(city='Tokyo')
Tool 실행 결과 (날씨 정보): 맑음, 28°C

--- 4단계: 최종 답변 생성 ---

===== 수동 Agent의 최종 답변 (성공) =====
일본의 수도인 도쿄의 현재 날씨는 맑고 기온은 28°C입니다.


> **고찰**: 성공했습니다! 하지만 여전히 LLM 호출을 4번이나 하고, 중간 결과를 `capital_info`, `capital_city` 같은 변수에 계속 저장하고 넘겨줘야 했습니다. 코드가 길고 복잡하며, 단계가 늘어날수록 관리의 어려움이 커집니다.
>
> **더 큰 문제는, 이 코드가 항상 성공하지 않는다는 것입니다.** 이제 이 코드가 얼마나 쉽게 망가지는지 확인해 보겠습니다.

#### 3-2. 실패 시나리오: 왜 수동 구현은 불안정한가?

아래 사례는 LLM의 성능과 무관하게 **100% 재현 가능한 실패**이며, 수동 구현의 본질적인 취약점을 보여줍니다.

##### 실패 사례 1: 예상치 못한 결과로 인한 코드 에러 (`TypeError`)
만약 첫 번째 Tool 검색이 실패하여 중간 결과로 `None`을 얻게 되면 어떻게 될까요?

In [10]:
print("---  실패 시나리오 1: 코드 레벨 에러 ---")

# --- 1단계: 수도 검색 (성공) --- #
capital_info = "일본의 수도는 도쿄입니다."
print(f"1단계 결과: {capital_info}")

# --- 2단계: 도시 이름 추출 시도... 하지만 결과를 강제로 None으로 조작! --- #
print("\n--- 2단계: 도시 이름 추출 (결과를 None으로 조작) ---")
capital_city = None
print(f"추출된 도시 이름 (조작됨): {capital_city}")

# --- 3단계: None 값으로 날씨 검색 시도 --- #
print("\n--- 3단계: 날씨 정보 검색 시도 ---")
try:
    #  다음 코드는 capital_city가 None이므로 에러를 발생시킵니다.
    messages = [{"role": "user", "content": f"{capital_city}의 현재 날씨 알려줘."}]
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
    tool_call = response.choices[0].message.tool_calls[0]
    weather_info = available_functions[tool_call.function.name](**json.loads(tool_call.function.arguments))
except Exception as e:
    print(f"\n 에러 발생! ")
    print(f"LLM API 또는 Tool 호출 중 다음과 같은 에러가 발생했습니다: \n{e}")
    print("\n[분석] 중간 결과(capital_city)가 None이 되자, 다음 단계의 코드가 이를 처리하지 못하고 시스템 전체가 멈췄습니다. 이것이 수동 구현의 가장 큰 취약점입니다.")

---  실패 시나리오 1: 코드 레벨 에러 ---
1단계 결과: 일본의 수도는 도쿄입니다.

--- 2단계: 도시 이름 추출 (결과를 None으로 조작) ---
추출된 도시 이름 (조작됨): None

--- 3단계: 날씨 정보 검색 시도 ---

 에러 발생! 
LLM API 또는 Tool 호출 중 다음과 같은 에러가 발생했습니다: 
'NoneType' object is not subscriptable

[분석] 중간 결과(capital_city)가 None이 되자, 다음 단계의 코드가 이를 처리하지 못하고 시스템 전체가 멈췄습니다. 이것이 수동 구현의 가장 큰 취약점입니다.


##### 실패 사례 2: 잘못된 Tool 선택으로 인한 논리적 실패
이번에는 코드 에러는 없지만, LLM이 잘못된 Tool을 선택하여 사용자가 원하는 답을 주지 못하는 '논리적 실패'입니다.

In [11]:
print("---  실패 시나리오 2: 논리적 실패 ---")

#  함정이 있는 질문: '날씨'와 '역사'를 동시에 질문
question = "도쿄의 현재 날씨와 역사에 대해 알려줘."
messages = [{"role": "user", "content": question}]
print(f"질문: {question}")

# --- LLM의 Tool 선택 --- #
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
tool_call = response.choices[0].message.tool_calls[0]
chosen_function_name = tool_call.function.name
function_args = json.loads(tool_call.function.arguments)

print(f"\nLLM의 선택: '{chosen_function_name}' (인자: {function_args})")

# --- 선택된 Tool 실행 --- #
result = available_functions[chosen_function_name](**function_args)

print(f"\n===== Agent의 답변 (논리적 실패) =====")
print(result)
print("\n[분석] Agent는 두 가지 질문 중 하나만 처리할 수 있는 Tool을 선택했습니다. 날씨를 물어보면 역사를 놓치고, 역사를 물어보면 날씨를 놓칩니다. 한 번에 하나의 Tool만 호출하는 단순한 로직의 한계입니다.")

---  실패 시나리오 2: 논리적 실패 ---
질문: 도쿄의 현재 날씨와 역사에 대해 알려줘.

LLM의 선택: 'get_current_weather' (인자: {'city': 'Tokyo'})
[Tool 실행] get_current_weather(city='Tokyo')

===== Agent의 답변 (논리적 실패) =====
맑음, 28°C

[분석] Agent는 두 가지 질문 중 하나만 처리할 수 있는 Tool을 선택했습니다. 날씨를 물어보면 역사를 놓치고, 역사를 물어보면 날씨를 놓칩니다. 한 번에 하나의 Tool만 호출하는 단순한 로직의 한계입니다.


---

### ✨ 4. LangChain을 통한 해결

이번에는 위에서 겪었던 모든 고통스러운 과정(**성공 시나리오의 복잡함, 실패 시나리오의 불안정성**)을 **LangChain의 `AgentExecutor`**가 어떻게 해결하는지 보겠습니다.

In [12]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import tool
from langchain import hub

In [13]:
# 1. LangChain이 사용할 Tool 정의 (기존 함수에 @tool 데코레이터만 붙이면 끝!)

@tool
def search_wikipedia(query: str) -> str:
    """위키백과에서 정보를 검색하여 요약합니다."""
    print(f"[Tool 실행] search_wikipedia(query='{query}')")
    try:
        wikipedia.set_lang("ko")
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"에러: {e}"

@tool
def get_current_weather(city: str) -> str:
    """특정 도시의 현재 날씨를 알려줍니다."""
    print(f"[Tool 실행] get_current_weather(city='{city}')")
    if "도쿄" in city or "Tokyo" in city:
        return "맑음, 28°C"
    elif "서울" in city or "Seoul" in city:
        return "흐림, 24°C"
    else:
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."

tools = [search_wikipedia, get_current_weather]

# 2. LLM, 프롬프트, Agent, 실행기 준비
llm = ChatOpenAI(model=MODEL, temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
# 3. Agent 실행! (단 한 줄의 코드로 모든 과정을 처리)
print("\n===== LangChain Agent 실행 =====")
question = "일본의 수도는 어디인지 위키백과에서 찾고, 그 도시의 현재 날씨를 알려줘."
result = agent_executor.invoke({"input": question})


===== LangChain Agent 실행 =====


> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': '일본의 수도'}`


[Tool 실행] search_wikipedia(query='일본의 수도')
현재의 일본의 수도는 도쿄이다. 이 문서는 역사속 일본의 수도(日本の首都)에 관해 기술한 것이다.
Invoking: `get_current_weather` with `{'city': 'Tokyo'}`


[Tool 실행] get_current_weather(city='Tokyo')
맑음, 28°C일본의 수도는 도쿄입니다. 현재 도쿄의 날씨는 맑고, 기온은 28°C입니다.

> Finished chain.


In [15]:
print("\n===== LangChain Agent의 최종 답변 =====")
print(result['output'])


===== LangChain Agent의 최종 답변 =====
일본의 수도는 도쿄입니다. 현재 도쿄의 날씨는 맑고, 기온은 28°C입니다.


---

### 🆚 5. 비교 및 정리

| 항목 | 수동 구현 | LangChain |
|:--- |:--- |:--- |
| **코드 길이** | 매우 김 (LLM 호출 4번 + 예외 처리) | 매우 짧음 (invoke 1번) |
| **상태 관리** | 직접 변수에 저장 (복잡, **에러에 취약**) | 자동 (Agent 내부 Scratchpad, **안정적**) |
| **Tool 선택** | 한 번에 하나만 가능 | 여러 Tool을 순차적으로 자동 선택 |
| **과정 추적** | print문으로 직접 찍어야 함 | `verbose=True`로 '생각의 사슬' 확인 가능 |

**핵심 결론**: Agent의 동작 원리를 이해하기 위해 수동 구현을 경험하는 것은 중요하지만, 실제 제품을 만들 때는 에러 처리, 상태 관리, 확장성 등 모든 면에서 **LangChain과 같은 프레임워크를 사용하는 것이 필수적**입니다.

---

### 🚀 6. 2주차 과제 안내

**🎯 목표**: 오늘 배운 LangChain `AgentExecutor`를 활용하여, 새로운 Tool 조합으로 문제를 직접 해결해 봅니다.

**📝 내용**:
1.  **새로운 Tool 만들기**: 두 숫자를 더하는 `add(a: int, b: int)` Tool과, 두 숫자를 곱하는 `multiply(a: int, b: int)` Tool을 만드세요.
2.  **Agent 만들기**: 직접 만든 `add`와 `multiply` Tool을 사용하는 LangChain Agent를 만드세요.
3.  **문제 해결**: 생성한 Agent에게 **"5에 7을 더한 값에 3을 곱해줘"** 라고 질문하고, 최종 답변을 얻는 전체 과정을 코드로 구현하세요.

**✅ 제출물**: 전체 실행 코드가 담긴 Jupyter Notebook (`.ipynb`) 파일